In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
REV_DIR = os.getcwd()
RES_DIR = os.path.dirname(REV_DIR)
ROOT_DIR = os.path.dirname(RES_DIR)
sys.path.append(ROOT_DIR)

AUROC의 t-test 비교

In [16]:
import mlflow
from sklearn.metrics import auc, roc_curve

from results.revision.revision_utils import  TRACKING_URI, RF_RUN_ID, BOOSTRAP_RUN_ID
from utils.serialization_ops import load_pickle
mlflow.set_tracking_uri(TRACKING_URI)

path = mlflow.artifacts.download_artifacts("mlflow-artifacts:/6/{run_id}/artifacts/fold_result.pickle".format(run_id=RF_RUN_ID))
fold_results_rf = load_pickle(path)

all_aurocs_rf = list()
n_folds = len(fold_results_rf["test_ids"])
for fold in range(n_folds):
    fold_bag_y_trues = fold_results_rf["fold_bag_y_trues"][fold]
    fold_bag_y_probs = fold_results_rf["fold_bag_y_probs"][fold]
    fpr, tpr, _ = roc_curve(fold_bag_y_trues, fold_bag_y_probs)
    all_aurocs_rf.append(auc(fpr, tpr))


path = mlflow.artifacts.download_artifacts("mlflow-artifacts:/6/{run_id}/artifacts/fold_result.pickle".format(run_id=BOOSTRAP_RUN_ID))
fold_results_mil = load_pickle(path)
n_folds = len(fold_results_mil["test_ids"])
all_aurocs_mil = list()
for fold in range(n_folds):
    fold_bag_y_trues = fold_results_mil["fold_bag_y_trues"][fold]
    fold_bag_y_probs = fold_results_mil["fold_bag_y_probs"][fold]
    fpr, tpr, _ = roc_curve(fold_bag_y_trues, fold_bag_y_probs)
    all_aurocs_mil.append(auc(fpr, tpr))

In [12]:
from results.revision.revision_utils import cohen_d
from scipy.stats import ttest_ind

t_stat, p_value = ttest_ind(all_aurocs_rf, all_aurocs_mil, equal_var=False)

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

d = cohen_d(all_aurocs_rf, all_aurocs_mil)
print(f"Cohen's d: {d}")

t-statistic: 1.9345708289916295
p-value: 0.06272168019574402
Cohen's d: 0.3780305083088606


SNV only case

In [15]:
from utils.serialization_ops import load_pickle
from results.revision.revision_utils import DATA_PATH
from core.data_model import PatientDataSet

patient_dataset:PatientDataSet = load_pickle(DATA_PATH)

In [52]:
len(test_ids)

4633

In [53]:
max(test_indices)

4630

In [ ]:
fold_

In [55]:
len(fold_results_rf["fold_instance_y_trues"][fold_idx])

1629

In [ ]:
import numpy as np
from results.revision.revision_utils import get_snv_only_case
from core.metric import Metric
ks = [1, 2, 3, 4, 5, 10, 15, 20, 100]
performance = np.ones((5, len(ks)))

for k_idx, k in enumerate(ks):
    for fold_idx in range(5):
        test_ids = fold_results_rf["test_ids"][fold_idx]
        snv_only_ids = get_snv_only_case(patient_dataset[test_ids])  
        
        test_indices = [test_ids.index(snv_id) for snv_id in snv_only_ids]

        for _id in test_indices:
            a = fold_results_rf["fold_instance_y_trues"][fold_idx][_id]
        instance_y_trues = [fold_results_rf["fold_instance_y_trues"][fold_idx][_id] for _id in test_indices]
        instance_y_probs = [fold_results_rf["fold_instance_y_trues"][fold_idx][_id] for _id in test_indices]
        topk = Metric.topk_recall(
            bag_labels=np.ones(len(instance_y_probs)), 
            instance_probs=instance_y_probs, 
            instance_labels=instance_y_trues, 
            k=k
        )
    
        performance[fold_idx, k_idx] = topk

IndexError: list index out of range

In [49]:
np.stack(fold_results_rf["fold_instance_y_trues"][i])

ValueError: all input arrays must have the same shape

In [43]:
len(snv_only_ids)

7769